In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd

In [88]:
dataset = pd.read_excel('/content/drive/MyDrive/code/تحلیل احساسات/train_digikala.xlsx')
dataset.head()

,text,plo
0,توی مشخصات کالا در قسمت اقلام همراه چیزی دربار...,1
1,بسیار عالی و باکیفیت هست و ارزش خرید داره,1
2,به توصیه دوستان برای اولین بار از دیجی کالا خر...,1
3,خیلی گوشیه خوبیه,1
4,فوق العاده,1


In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(dataset['text'])


In [90]:
from sklearn.model_selection import cross_val_score

In [10]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import GlobalMaxPooling1D,Flatten,Dropout,Dense, GRU, Embedding, LSTM,Conv1D,BatchNormalization
from tensorflow.keras.layers import Dropout,GlobalMaxPooling1D,Conv1D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [96]:
stop_words =['از','به','با','برای','که',
            ]
#پیش پردازش
import re
def preprocess_text(sentence):
    # Removing multiple spaces
    sentence = re.sub(r'@\S+', '', sentence)
    sentence = re.sub(r'!\S+', '', sentence)
    sentence = re.sub(r'؟\S+', '', sentence)
    sentence = re.sub(r'[.]', ' ', sentence)
    sentence = re.sub(r'[/]', '', sentence)
    sentence = re.sub(r'[،]', ' ', sentence)
    sentence = re.sub(r'[؛]', '', sentence)
    sentence = sentence.split()
    sentence =[word for word in sentence if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence
def ReplacetwoMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1", s)
dataset['clean_text'] = dataset['text'].apply(ReplacetwoMore)

X = []
sentences = list(dataset["clean_text"])
for sen in sentences:
    X.append(preprocess_text(sen))    

In [97]:
target = dataset['plo']
data = dataset['clean_text']
x_train = data
y_train= target
print(x_train.shape)

(2000,)


In [98]:
import nltk
import numpy as np
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
print ('number unique word:',num_unique_word)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
number unique word: 5398


In [99]:
max_tokens = 58

In [100]:
print(dataset.groupby(["plo"]).size())

plo
0    1000
1    1000
dtype: int64


In [101]:

num_words = num_unique_word
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)

x_train_tokens = tokenizer.texts_to_sequences(x_train)

print(x_train[100])
print(x_train_tokens[100])
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding='post')


idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
print('train shape',x_train_pad.shape)



به شدت پیشنهاد میکنم من این گوشی رو یکماه که دارم فوق العاده ازش راضی ام به نظرات منفی اصلا توچه نکنید این گوشی هیچی براتون کم‌نمیزاره اگر براتون تایپ سی و فست شارژ مهم نیست خریدشو حتما پیشنهاد میکنم
[3, 269, 26, 37, 6, 2, 11, 10, 957, 4, 81, 90, 132, 58, 41, 160, 3, 392, 606, 14, 2435, 112, 2, 11, 328, 475, 2436, 141, 475, 860, 233, 1, 1095, 83, 363, 31, 861, 91, 26, 37]
train shape (2000, 58)


In [17]:
from sklearn.model_selection import KFold
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)

In [84]:
from scipy import stats as s


In [29]:
from statistics import mode

In [102]:
kfold = KFold(n_splits=5, shuffle=True)
acc_per_fold_GRU = []
per_per_fold_GRU = []
rec_per_fold_GRU = []
f1_per_fold_GRU = []

acc_per_fold_c = []
per_per_fold_c = []
rec_per_fold_c = []
f1_per_fold_c = []

acc_per_fold_L= []
per_per_fold_L = []
rec_per_fold_L = []
f1_per_fold_L = []


acc_per_fold_L2= []
per_per_fold_L2 = []
rec_per_fold_L2 = []
f1_per_fold_L2 = []

acc_per_fold_av= []
per_per_fold_av = []
rec_per_fold_av = []
f1_per_fold_av = []
fold_no = 1
for train, test in kfold.split(x_train_pad, y_train):   
    embedding_size = 100
    model4 = Sequential()
    model4.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model4.add(Dropout(0.5))
    model4.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model4.add(BatchNormalization())
    model4.add(GlobalMaxPooling1D())
    model4.add(Dense(200, activation='relu'))
    model4.add(Dropout(0.5))
    model4.add(Dense(1,activation='sigmoid'))
    model4.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    ####################################
    embedding_size2 = 150
    model2 = Sequential()
    model2.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size2,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model2.add(Dropout(0.5))
    model2.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model2.add(BatchNormalization())
    model2.add(GlobalMaxPooling1D())
    model2.add(Dense(200, activation='relu'))
    model2.add(Dropout(0.5))
    model2.add(Dense(1,activation='sigmoid'))
    model2.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
  #############
    embedding_size3 = 200
    model5 = Sequential()
    model5.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size3,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model5.add(Dropout(0.5))
    model5.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model5.add(BatchNormalization())
    model5.add(GlobalMaxPooling1D())
    model5.add(Dense(200, activation='relu'))
    model5.add(Dropout(0.5))
    model5.add(Dense(1,activation='sigmoid'))
    model5.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    #############
    embedding_size4 = 250
    model6 = Sequential()
    model6.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size4,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model6.add(Dropout(0.5))
    model6.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model6.add(BatchNormalization())
    model6.add(GlobalMaxPooling1D())
    model6.add(Dense(200, activation='relu'))
    model6.add(Dropout(0.5))
    model6.add(Dense(1,activation='sigmoid'))
    model6.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])




    print('------------------------------------------------------------------------')
    print(f'CNN: Training for fold {fold_no} ...')
    print(" CNN WE 100")
    history = model4.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model4.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred=model4.predict(x_train_pad[test])
    y_pred = np.round(y_pred).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred)
    recall = recall_score(y_train[test], y_pred , average="binary")
    precision = precision_score(y_train[test], y_pred , average="binary")
    f1 = f1_score(y_train[test], y_pred, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'CNN 100: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_GRU.append(accuracy * 100)
    per_per_fold_GRU.append(recall * 100) 
    rec_per_fold_GRU.append(precision * 100) 
    f1_per_fold_GRU.append(f1 * 100) 







    print(" CNN WE 150")
    history2 = model2.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model2.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred2=model2.predict(x_train_pad[test])
    y_pred2 = np.round(y_pred2).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred2)
    recall = recall_score(y_train[test], y_pred2 , average="binary")
    precision = precision_score(y_train[test], y_pred2 , average="binary")
    f1 = f1_score(y_train[test], y_pred2, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'CNN 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_c.append(accuracy * 100)
    per_per_fold_c.append(recall * 100) 
    rec_per_fold_c.append(precision * 100) 
    f1_per_fold_c.append(f1 * 100) 
    



    print(" CNN WE 200")
    history3 = model5.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model5.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred3=model5.predict(x_train_pad[test])
    y_pred3 = np.round(y_pred3).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred3)
    recall = recall_score(y_train[test], y_pred3 , average="binary")
    precision = precision_score(y_train[test], y_pred3 , average="binary")
    f1 = f1_score(y_train[test], y_pred3, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'LSTM 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_L.append(accuracy * 100)
    per_per_fold_L.append(recall * 100) 
    rec_per_fold_L.append(precision * 100) 
    f1_per_fold_L.append(f1 * 100)



    print(" CNN WE 250")
    history4 = model6.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model6.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred4=model6.predict(x_train_pad[test])
    y_pred4 = np.round(y_pred4).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred4)
    recall = recall_score(y_train[test], y_pred4 , average="binary")
    precision = precision_score(y_train[test], y_pred4 , average="binary")
    f1 = f1_score(y_train[test], y_pred4, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'LSTM 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_L2.append(accuracy * 100)
    per_per_fold_L2.append(recall * 100) 
    rec_per_fold_L2.append(precision * 100) 
    f1_per_fold_L2.append(f1 * 100)  




    p1=model4.predict(x_train_pad[test])
    p1 = np.round(p1).astype(int)
    p2=model2.predict(x_train_pad[test])
    p2 = np.round(p2).astype(int)
    p3=model5.predict(x_train_pad[test])
    p3 = np.round(p3).astype(int)
    p4=model6.predict(x_train_pad[test])
    p4 = np.round(p4).astype(int)

    final_pred = np.array([])
    for i in range(0,len(x_train_pad[test])):
      final_pred = np.append(final_pred, s.mode([p1[i][0], p2[i][0], p3[i][0], p4[i][0]])[0]  )
    accuracy=accuracy_score(y_train[test], final_pred)
    recall = recall_score(y_train[test], final_pred , average="binary")
    precision = precision_score(y_train[test], final_pred , average="binary")
    f1 = f1_score(y_train[test], final_pred, average="binary")
    print("Max Voting")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))

    acc_per_fold_av.append(accuracy * 100)
    per_per_fold_av.append(recall * 100) 
    rec_per_fold_av.append(precision * 100) 
    f1_per_fold_av.append(f1 * 100) 


    fold_no = fold_no + 1

------------------------------------------------------------------------
CNN: Training for fold 1 ...
 CNN WE 100
Epoch 1/10
7/7 [==============================] - 2s 110ms/step - loss: 1.1298 - accuracy: 0.4819 - val_loss: 0.6916 - val_accuracy: 0.4975
Epoch 2/10
7/7 [==============================] - 0s 29ms/step - loss: 0.7702 - accuracy: 0.5656 - val_loss: 0.6896 - val_accuracy: 0.7250
Epoch 3/10
7/7 [==============================] - 0s 27ms/step - loss: 0.6535 - accuracy: 0.6281 - val_loss: 0.6886 - val_accuracy: 0.7050
Epoch 4/10
7/7 [==============================] - 0s 26ms/step - loss: 0.5482 - accuracy: 0.7287 - val_loss: 0.6862 - val_accuracy: 0.6200
Epoch 5/10
7/7 [==============================] - 0s 28ms/step - loss: 0.4437 - accuracy: 0.8269 - val_loss: 0.6776 - val_accuracy: 0.7700
Epoch 6/10
7/7 [==============================] - 0s 27ms/step - loss: 0.3523 - accuracy: 0.8644 - val_loss: 0.6648 - val_accuracy: 0.8325
Epoch 7/10
7/7 [==============================] - 0

In [103]:
print('------------------------------------------------------------------------')
print('CNN 100: Score per fold')
for i in range(0, len(acc_per_fold_GRU)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_GRU[i]}% - Recall: {rec_per_fold_GRU[i]}% - Precision: {per_per_fold_GRU[i]}% - F1: {f1_per_fold_GRU[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 100: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_GRU)} (+- {np.std(acc_per_fold_GRU)})')
print(f'> Recall: {np.mean(rec_per_fold_GRU)} (+- {np.std(rec_per_fold_GRU)})')
print(f'> Precision: {np.mean(per_per_fold_GRU)} (+- {np.std(per_per_fold_GRU)})')
print(f'> F1: {np.mean(f1_per_fold_GRU)} (+- {np.std(f1_per_fold_GRU)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 150: Score per fold')
for i in range(0, len(acc_per_fold_c)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_c[i]}% - Recall: {rec_per_fold_c[i]}% - Precision: {per_per_fold_c[i]}% - F1: {f1_per_fold_c[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 150: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_c)} (+- {np.std(acc_per_fold_c)})')
print(f'> Recall: {np.mean(rec_per_fold_c)} (+- {np.std(rec_per_fold_c)})')
print(f'> Precision: {np.mean(per_per_fold_c)} (+- {np.std(per_per_fold_c)})')
print(f'> F1: {np.mean(f1_per_fold_c)} (+- {np.std(f1_per_fold_c)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 200: Score per fold')
for i in range(0, len(acc_per_fold_L)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_L[i]}% - Recall: {rec_per_fold_L[i]}% - Precision: {per_per_fold_L[i]}% - F1: {f1_per_fold_L[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 200: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_L)} (+- {np.std(acc_per_fold_L)})')
print(f'> Recall: {np.mean(rec_per_fold_L)} (+- {np.std(rec_per_fold_L)})')
print(f'> Precision: {np.mean(per_per_fold_L)} (+- {np.std(per_per_fold_L)})')
print(f'> F1: {np.mean(f1_per_fold_L)} (+- {np.std(f1_per_fold_L)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 250: Score per fold')
for i in range(0, len(acc_per_fold_L2)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_L2[i]}% - Recall: {rec_per_fold_L2[i]}% - Precision: {per_per_fold_L2[i]}% - F1: {f1_per_fold_L2[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 250: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_L2)} (+- {np.std(acc_per_fold_L2)})')
print(f'> Recall: {np.mean(rec_per_fold_L2)} (+- {np.std(rec_per_fold_L2)})')
print(f'> Precision: {np.mean(per_per_fold_L2)} (+- {np.std(per_per_fold_L2)})')
print(f'> F1: {np.mean(f1_per_fold_L2)} (+- {np.std(f1_per_fold_L2)})')
print('------------------------------------------------------------------------')



print('-------------------------------Max Voting -------------------------------')
print('Max Voting: Score per fold')
for i in range(0, len(acc_per_fold_av)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_av[i]}% - Recall: {rec_per_fold_av[i]}% - Precision: {per_per_fold_av[i]}% - F1: {f1_per_fold_av[i]}%')
  
print('------------------------------------------------------------------------')
print('Max Voting: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_av)} (+- {np.std(acc_per_fold_av)})')
print(f'> Recall: {np.mean(rec_per_fold_av)} (+- {np.std(rec_per_fold_av)})')
print(f'> Precision: {np.mean(per_per_fold_av)} (+- {np.std(per_per_fold_av)})')
print(f'> F1: {np.mean(f1_per_fold_av)} (+- {np.std(f1_per_fold_av)})')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
CNN 100: Score per fold
------------------------------------------------------------------------
> Fold 1  - Accuracy: 89.5% - Recall: 85.86956521739131% - Precision: 90.80459770114942% - F1: 88.26815642458101%
------------------------------------------------------------------------
> Fold 2  - Accuracy: 90.75% - Recall: 91.74311926605505% - Precision: 91.32420091324201% - F1: 91.53318077803205%
------------------------------------------------------------------------
> Fold 3  - Accuracy: 91.75% - Recall: 92.11822660098522% - Precision: 91.66666666666666% - F1: 91.89189189189189%
------------------------------------------------------------------------
> Fold 4  - Accuracy: 92.0% - Recall: 91.53439153439153% - Precision: 91.53439153439153% - F1: 91.53439153439153%
------------------------------------------------------------------------
> Fold 5  - Accuracy: 92.75% - Recall: 94.25837320574163% - Precision: 92.056074

In [104]:
kfold = KFold(n_splits=10, shuffle=True)
acc_per_fold_GRU = []
per_per_fold_GRU = []
rec_per_fold_GRU = []
f1_per_fold_GRU = []

acc_per_fold_c = []
per_per_fold_c = []
rec_per_fold_c = []
f1_per_fold_c = []

acc_per_fold_L= []
per_per_fold_L = []
rec_per_fold_L = []
f1_per_fold_L = []


acc_per_fold_L2= []
per_per_fold_L2 = []
rec_per_fold_L2 = []
f1_per_fold_L2 = []

acc_per_fold_av= []
per_per_fold_av = []
rec_per_fold_av = []
f1_per_fold_av = []
fold_no = 1
for train, test in kfold.split(x_train_pad, y_train):   
    embedding_size = 100
    model4 = Sequential()
    model4.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model4.add(Dropout(0.5))
    model4.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model4.add(BatchNormalization())
    model4.add(GlobalMaxPooling1D())
    model4.add(Dense(200, activation='relu'))
    model4.add(Dropout(0.5))
    model4.add(Dense(1,activation='sigmoid'))
    model4.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    ####################################
    embedding_size2 = 150
    model2 = Sequential()
    model2.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size2,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model2.add(Dropout(0.5))
    model2.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model2.add(BatchNormalization())
    model2.add(GlobalMaxPooling1D())
    model2.add(Dense(200, activation='relu'))
    model2.add(Dropout(0.5))
    model2.add(Dense(1,activation='sigmoid'))
    model2.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
  #############
    embedding_size3 = 200
    model5 = Sequential()
    model5.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size3,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model5.add(Dropout(0.5))
    model5.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model5.add(BatchNormalization())
    model5.add(GlobalMaxPooling1D())
    model5.add(Dense(200, activation='relu'))
    model5.add(Dropout(0.5))
    model5.add(Dense(1,activation='sigmoid'))
    model5.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    #############
    embedding_size4 = 250
    model6 = Sequential()
    model6.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size4,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model6.add(Dropout(0.5))
    model6.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model6.add(BatchNormalization())
    model6.add(GlobalMaxPooling1D())
    model6.add(Dense(200, activation='relu'))
    model6.add(Dropout(0.5))
    model6.add(Dense(1,activation='sigmoid'))
    model6.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])




    print('------------------------------------------------------------------------')
    print(f'CNN: Training for fold {fold_no} ...')
    print(" CNN WE 100")
    history = model4.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model4.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred=model4.predict(x_train_pad[test])
    y_pred = np.round(y_pred).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred)
    recall = recall_score(y_train[test], y_pred , average="binary")
    precision = precision_score(y_train[test], y_pred , average="binary")
    f1 = f1_score(y_train[test], y_pred, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'CNN 100: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_GRU.append(accuracy * 100)
    per_per_fold_GRU.append(recall * 100) 
    rec_per_fold_GRU.append(precision * 100) 
    f1_per_fold_GRU.append(f1 * 100) 







    print(" CNN WE 150")
    history2 = model2.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model2.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred2=model2.predict(x_train_pad[test])
    y_pred2 = np.round(y_pred2).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred2)
    recall = recall_score(y_train[test], y_pred2 , average="binary")
    precision = precision_score(y_train[test], y_pred2 , average="binary")
    f1 = f1_score(y_train[test], y_pred2, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'CNN 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_c.append(accuracy * 100)
    per_per_fold_c.append(recall * 100) 
    rec_per_fold_c.append(precision * 100) 
    f1_per_fold_c.append(f1 * 100) 
    



    print(" CNN WE 200")
    history3 = model5.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model5.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred3=model5.predict(x_train_pad[test])
    y_pred3 = np.round(y_pred3).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred3)
    recall = recall_score(y_train[test], y_pred3 , average="binary")
    precision = precision_score(y_train[test], y_pred3 , average="binary")
    f1 = f1_score(y_train[test], y_pred3, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'LSTM 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_L.append(accuracy * 100)
    per_per_fold_L.append(recall * 100) 
    rec_per_fold_L.append(precision * 100) 
    f1_per_fold_L.append(f1 * 100)



    print(" CNN WE 250")
    history4 = model6.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model6.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred4=model6.predict(x_train_pad[test])
    y_pred4 = np.round(y_pred4).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred4)
    recall = recall_score(y_train[test], y_pred4 , average="binary")
    precision = precision_score(y_train[test], y_pred4 , average="binary")
    f1 = f1_score(y_train[test], y_pred4, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'LSTM 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_L2.append(accuracy * 100)
    per_per_fold_L2.append(recall * 100) 
    rec_per_fold_L2.append(precision * 100) 
    f1_per_fold_L2.append(f1 * 100)  




    p1=model4.predict(x_train_pad[test])
    p1 = np.round(p1).astype(int)
    p2=model2.predict(x_train_pad[test])
    p2 = np.round(p2).astype(int)
    p3=model5.predict(x_train_pad[test])
    p3 = np.round(p3).astype(int)
    p4=model6.predict(x_train_pad[test])
    p4 = np.round(p4).astype(int)

    final_pred = np.array([])
    for i in range(0,len(x_train_pad[test])):
      final_pred = np.append(final_pred, s.mode([p1[i][0], p2[i][0], p3[i][0], p4[i][0]])[0]  )
    accuracy=accuracy_score(y_train[test], final_pred)
    recall = recall_score(y_train[test], final_pred , average="binary")
    precision = precision_score(y_train[test], final_pred , average="binary")
    f1 = f1_score(y_train[test], final_pred, average="binary")
    print("Max Voting")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))

    acc_per_fold_av.append(accuracy * 100)
    per_per_fold_av.append(recall * 100) 
    rec_per_fold_av.append(precision * 100) 
    f1_per_fold_av.append(f1 * 100) 


    fold_no = fold_no + 1

------------------------------------------------------------------------
CNN: Training for fold 1 ...
 CNN WE 100
Epoch 1/10
8/8 [==============================] - 2s 84ms/step - loss: 1.0360 - accuracy: 0.5044 - val_loss: 0.6897 - val_accuracy: 0.5500
Epoch 2/10
8/8 [==============================] - 0s 24ms/step - loss: 0.8113 - accuracy: 0.5483 - val_loss: 0.6908 - val_accuracy: 0.4650
Epoch 3/10
8/8 [==============================] - 0s 22ms/step - loss: 0.6502 - accuracy: 0.6289 - val_loss: 0.6883 - val_accuracy: 0.7700
Epoch 4/10
8/8 [==============================] - 0s 21ms/step - loss: 0.5550 - accuracy: 0.7217 - val_loss: 0.6863 - val_accuracy: 0.8100
Epoch 5/10
8/8 [==============================] - 0s 21ms/step - loss: 0.4886 - accuracy: 0.7778 - val_loss: 0.6819 - val_accuracy: 0.6550
Epoch 6/10
8/8 [==============================] - 0s 21ms/step - loss: 0.3948 - accuracy: 0.8339 - val_loss: 0.6710 - val_accuracy: 0.8500
Epoch 7/10
8/8 [==============================] - 0s

In [106]:
print('------------------------------------------------------------------------')
print('CNN 100: Score per fold')
for i in range(0, len(acc_per_fold_GRU)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_GRU[i]}% - Recall: {rec_per_fold_GRU[i]}% - Precision: {per_per_fold_GRU[i]}% - F1: {f1_per_fold_GRU[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 100: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_GRU)} (+- {np.std(acc_per_fold_GRU)})')
print(f'> Recall: {np.mean(rec_per_fold_GRU)} (+- {np.std(rec_per_fold_GRU)})')
print(f'> Precision: {np.mean(per_per_fold_GRU)} (+- {np.std(per_per_fold_GRU)})')
print(f'> F1: {np.mean(f1_per_fold_GRU)} (+- {np.std(f1_per_fold_GRU)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 150: Score per fold')
for i in range(0, len(acc_per_fold_c)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_c[i]}% - Recall: {rec_per_fold_c[i]}% - Precision: {per_per_fold_c[i]}% - F1: {f1_per_fold_c[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 150: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_c)} (+- {np.std(acc_per_fold_c)})')
print(f'> Recall: {np.mean(rec_per_fold_c)} (+- {np.std(rec_per_fold_c)})')
print(f'> Precision: {np.mean(per_per_fold_c)} (+- {np.std(per_per_fold_c)})')
print(f'> F1: {np.mean(f1_per_fold_c)} (+- {np.std(f1_per_fold_c)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 200: Score per fold')
for i in range(0, len(acc_per_fold_L)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_L[i]}% - Recall: {rec_per_fold_L[i]}% - Precision: {per_per_fold_L[i]}% - F1: {f1_per_fold_L[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 200: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_L)} (+- {np.std(acc_per_fold_L)})')
print(f'> Recall: {np.mean(rec_per_fold_L)} (+- {np.std(rec_per_fold_L)})')
print(f'> Precision: {np.mean(per_per_fold_L)} (+- {np.std(per_per_fold_L)})')
print(f'> F1: {np.mean(f1_per_fold_L)} (+- {np.std(f1_per_fold_L)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 250: Score per fold')
for i in range(0, len(acc_per_fold_L2)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_L2[i]}% - Recall: {rec_per_fold_L2[i]}% - Precision: {per_per_fold_L2[i]}% - F1: {f1_per_fold_L2[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 250: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_L2)} (+- {np.std(acc_per_fold_L2)})')
print(f'> Recall: {np.mean(rec_per_fold_L2)} (+- {np.std(rec_per_fold_L2)})')
print(f'> Precision: {np.mean(per_per_fold_L2)} (+- {np.std(per_per_fold_L2)})')
print(f'> F1: {np.mean(f1_per_fold_L2)} (+- {np.std(f1_per_fold_L2)})')
print('------------------------------------------------------------------------')



print('-------------------------------Max Voting -------------------------------')
print('Max Voting: Score per fold')
for i in range(0, len(acc_per_fold_av)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_av[i]}% - Recall: {rec_per_fold_av[i]}% - Precision: {per_per_fold_av[i]}% - F1: {f1_per_fold_av[i]}%')
  
print('------------------------------------------------------------------------')
print('Max Voting: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_av)} (+- {np.std(acc_per_fold_av)})')
print(f'> Recall: {np.mean(rec_per_fold_av)} (+- {np.std(rec_per_fold_av)})')
print(f'> Precision: {np.mean(per_per_fold_av)} (+- {np.std(per_per_fold_av)})')
print(f'> F1: {np.mean(f1_per_fold_av)} (+- {np.std(f1_per_fold_av)})')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
CNN 100: Score per fold
------------------------------------------------------------------------
> Fold 1  - Accuracy: 90.0% - Recall: 82.72727272727273% - Precision: 98.91304347826086% - F1: 90.0990099009901%
------------------------------------------------------------------------
> Fold 2  - Accuracy: 89.5% - Recall: 86.95652173913044% - Precision: 94.33962264150944% - F1: 90.49773755656109%
------------------------------------------------------------------------
> Fold 3  - Accuracy: 89.5% - Recall: 85.84905660377359% - Precision: 93.81443298969072% - F1: 89.65517241379311%
------------------------------------------------------------------------
> Fold 4  - Accuracy: 89.0% - Recall: 82.45614035087719% - Precision: 97.91666666666666% - F1: 89.52380952380952%
------------------------------------------------------------------------
> Fold 5  - Accuracy: 93.0% - Recall: 93.45794392523365% - Precision: 93.4579439252